In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import random
from tqdm import tqdm
import pandas as pd
import json

import torch
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import BertConfig, BertTokenizer
from model.bertforsequence import *

from config.config import *
from utils.load_data import *
from utils.utils import *


I1220 02:36:36.628856 140322404288320 file_utils.py:39] PyTorch version 1.1.0 available.
I1220 02:36:36.880711 140322404288320 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Devices: cuda


In [3]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=DO_LOWER_CASE)
test_dataset = load_and_cache_examples(tokenizer, TEST_FILE) 

I1220 02:36:38.181724 140322404288320 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/tracy/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


2019-12-20 02:36:38,219 - load_data.py - INFO - Loading features from cached file /home/tracy/Kaggle/TF2_QA/data/cached_test_bert_320


I1220 02:36:38.219702 140322404288320 load_data.py:168] Loading features from cached file /home/tracy/Kaggle/TF2_QA/data/cached_test_bert_320


In [4]:
TRANSFORMER_MODEL = BertForSequenceClassification
stored_dir = "runs/bert_base"

config = BertConfig.from_pretrained(stored_dir, num_labels=LABEL_SIZE, finetuning_task="zalo")
model = TRANSFORMER_MODEL.from_pretrained(stored_dir, from_tf=False, config=config)

I1220 02:36:41.476251 140322404288320 configuration_utils.py:148] loading configuration file runs/bert_base/config.json
I1220 02:36:41.478592 140322404288320 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "zalo",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1220 02:36:41.479749 140322404288320 modeling_utils.py:334] loading weights file runs/bert_base/pytorch_model.bin


In [5]:
model.to(DEVICE)
model.eval()

eval_sampler = SequentialSampler(test_dataset)
eval_dataloader = DataLoader(test_dataset, sampler=eval_sampler, batch_size=BATCH_SIZE)

epoch_iterator = tqdm(eval_dataloader, desc="Iteration", disable=False)
preds = None

for step, batch in enumerate(epoch_iterator):
    model.eval()
    batch = tuple(t.to(DEVICE) for t in batch)

    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2] if MODEL_TYPE in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                  'labels':         batch[3]}

        try:
            model.batch_size = len(batch[0])
            model.lstm_hidden_1 = model.init_hidden()
            model.lstm_hidden_2 = model.init_hidden()
        except:
            pass

        outputs = model(**inputs)
        _, logits = outputs[:2]
    if preds is None:
        preds = logits.detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

Iteration: 100%|██████████| 5646/5646 [13:39<00:00,  6.89it/s]


In [14]:
argmax_preds = [np.argmax(p) for p in preds]

In [16]:
from collections import Counter
Counter(argmax_preds)

Counter({0: 22389, 1: 22584, 3: 169, 2: 21})

In [20]:
df = pd.read_csv("/home/tracy/Kaggle/TF2_QA/data/test_1.csv", sep="\t")

In [22]:
df["label"] = argmax_preds

In [24]:
df.to_csv("test_predict_1.csv", index=False)

,example_id,question,text,start_token,end_token,label
0,-1220107454853145579,who is the south african high commissioner in ...,"<Table> <Tr> <Th_colspan=""2""> High Commission ...",18,136,0
1,-1220107454853145579,who is the south african high commissioner in ...,"<Tr> <Th_colspan=""2""> High Commission of South...",19,30,0
2,-1220107454853145579,who is the south african high commissioner in ...,<Tr> <Th> Location </Th> <Td> Trafalgar Square...,34,45,0
3,-1220107454853145579,who is the south african high commissioner in ...,<Tr> <Th> Address </Th> <Td> Trafalgar Square ...,45,59,0
4,-1220107454853145579,who is the south african high commissioner in ...,<Tr> <Th> Coordinates </Th> <Td> 51 ° 30 ′ 30 ...,59,126,0
...,...,...,...,...,...,...
45158,-3564167666631898036,song hit me with your best shot by pat benatar,<Ul> <Li> The song plays during the Disney Cal...,1370,1393,1
45159,-3564167666631898036,song hit me with your best shot by pat benatar,<Li> The song plays during the Disney Californ...,1371,1392,1
45160,-3564167666631898036,song hit me with your best shot by pat benatar,<P> Other artists who have performed the song ...,1403,1436,1
45161,-3564167666631898036,song hit me with your best shot by pat benatar,<P> Fictional music group Alvin and the Chipmu...,1436,1456,1


In [37]:
list_example_id = df["example_id"]
list_example_id = list(set(list_example_id))

In [40]:
for i in list_example_id:
    sample_df = df[(df["example_id"] == i) & (df["label"] != 0)]
    sample_df = sample_df.reset_index()
    if len(sample_df) > 0:
        print("{}_long,{}:{}".format(i, sample_df["start_token"][0], sample_df["end_token"][0]))
    short_

-7473086642300315642_long,37:47
8808974242947452932_long,44:56
-8752372642178983917_long,33:43
8622726872540768274_long,254:452
-896667357940908006_long,191:415
4332907778765488156_long,65:75
-430548376093792225_long,241:251
-1883092661126100958_long,96:107
7517618431482920992_long,85:108
-697706369125523422_long,169:187
-1636829004676923357_long,62:79
-6218651186569760726_long,39:49
-4862695314536943573_long,75:113
-2225412054239195091_long,194:200
-5902735205614223305_long,483:497
-25395840162770885_long,43:2295
4548074189750870077_long,818:865
2368303174387226686_long,82:92
8009244943647758396_long,56:200
-7899881196616738741_long,21:130
6244104306165846098_long,66:82
8847717112932520020_long,55:134
7537193362496034910_long,210:326
7294303183050707045_long,43:53
3203035384644528250_long,50:138
-5231116141547962238_long,309:319
2376173919358511235_long,660:699
-969876909610971000_long,238:349
-6804121595151966068_long,9:15
-657449628892962677_long,162:225
5478445368933023882_long,43:

-2014171866821038290_long,31:41
6903455298740107054_long,42:52
3680817750529140547_long,1561:1810
-6808782971978873013_long,84:100
4026141052159471435_long,53:69
3977985659376901964_long,59:133
-3652094434423830699_long,146:180
-2320963541249468565_long,94:109
-6739468307275589778_long,161:844
885665536597931885_long,403:523
1027713819871831920_long,193:238
-4328558629320992906_long,144:166
4488854086541059964_long,111:124
7909023684769079165_long,41:67
4026611122767529872_long,641:808
2935867962256611217_long,124:150
4774513718149377942_long,19:149
-7116062623163449442_long,246:321
4977367927590680491_long,95:106
7919080561402978223_long,113:177
4153219118929483698_long,390:472
8980113463461562294_long,46:55
418890410382116795_long,495:609
-7537447137505972289_long,210:326
-2032714444673581121_long,78:148
6026862874448345027_long,135:145
-6714088971745431608_long,60:75
-5249710812891134006_long,42:57
4739159254043531211_long,12:65
-8161758226997940269_long,40:72
-1219507076732106786_l